# OpenStreetMap example

In this example, we download a road network from OSM using the OSMNx package, and then process the result, resulting in a RouteE Compass network dataset.

### requirements

To download an open street maps dataset, you'll need a couple of extra dependencies: [osmnx](https://osmnx.readthedocs.io/en/stable/) and [tomllib](https://docs.python.org/3/library/tomllib.html) (Python>3.11) or [toml](https://pypi.org/project/toml/) (Python<3.11). 

You can install these doing: 

```bash
pip install nrel.routee.compass[all]
```

Note, this also includes some plotting dependencies for visualizing results. If you just want the osm based dependencies you can do:

```bash
pip install nrel.routee.compass[osm]
```

In [1]:
import osmnx as ox
from nrel.routee.compass.io import generate_compass_dataset
from nrel.routee.compass import CompassApp
from pathlib import Path
import os

Here we set the absolute path to output directory where you want the dataset collected:

In [2]:
output_directory = Path("golden_co")              

In this example will load in the road network that covers Golden, Colorado, as a small example but this workflow will work with any osmnx graph.

Osmnx provides many graph download operations, any are acceptable.

In [3]:
g = ox.graph_from_place("Golden, Colorado, USA", network_type="drive")  

In [4]:
ox.plot_graph_folium(g)

/var/folders/nn/7wz02d9945sc16d9bd1c2fq8pmzlw0/T/ipykernel_74857/1273448676.py:1: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  ox.plot_graph_folium(g)


Now, we call the `generate_compass_dataset` function which will convert the osmnx graph into files that are compatible with routee-compass.

The network speeds arguments can also be provided here, see documentation on `generate_compass_dataset` or `osmnx.add_edge_speeds` for details.

In [5]:
generate_compass_dataset(g, output_directory)      

In order for the path arguments to work in the example TOML file, we change this Python environment's working directory to the dataset directory

In [6]:
os.chdir(output_directory)                         

If you take a look in the directory where the compass files have been written, you'll notice some `.toml` files like: `osm_default_energy.toml`. 
These are configurations for the compass application. Here's what it might look like:

```toml
parallelism = 2

[graph]
edge_list_csv = "edges-compass.csv.gz"
vertex_list_csv = "vertices-compass.csv.gz"
verbose = true

[traversal]
type = "speed_grade_energy_model"
model_type = "smartcore"
speed_table_path = "edges-posted-speed-enumerated.txt.gz"
energy_model_path = "2016_TOYOTA_Camry_4cyl_2WD.bin"
ideal_energy_rate = 0.02857142857
speed_table_speed_unit = "kilometers_per_hour"
energy_model_speed_unit = "miles_per_hour"
energy_model_grade_unit = "decimal"
energy_model_energy_rate_unit = "gallons_gasoline_per_mile"
output_time_unit = "minutes"
output_distance_unit = "miles"

[plugin]
[[plugin.input_plugins]]
type = "vertex_rtree"
vertices_file = "vertices-compass.csv.gz"

[[plugin.output_plugins]]
type = "summary"

[[plugin.output_plugins]]
type = "traversal"
route = "geo_json"
tree = "geo_json"
geometry_file = "edges-geometries-enumerated.txt.gz"

[[plugin.output_plugins]]
type = "uuid"
uuid_file = "vertices-uuid-enumerated.txt.gz"
```

This defines the parameters for the routing engine.
See the documentation section of the config for more information.

Now we can load the application from one of our config files. 
We'll pick `osm_default_energy.toml` which will use a Toyota Camry energy model for finding the least energy route.

In [7]:
app = CompassApp.from_config_file("osm_default_energy.toml")  

uuid file: 100%|██████████| 746/746 [00:00<00:00, 3789456.50it/s]it/s]

Now we have an application that we can run queries against.
To demonstrate, we'll route between two locations in Golden, CO utilzing the grid search input plugin to run two separate searches. 
The energy cost coefficient indicates how much we should factor energy into our route search, 0.0 indicating that we should not factor in energy at all (shortest time route) and 1.0 indicating that we should only factor energy in (least energy route)

In [8]:
query = {
    "origin_name": "Government Center Station",
    "destination_name": "Cannonball Creek Brewery",
    "origin_x": -105.200146,
    "origin_y": 39.726570,
    "destination_x": -105.234964,
    "destination_y": 39.768477,
    "grid_search": {
        "energy_cost_coefficient": [0.0, 1.0]
    }
}

In [9]:
results = app.run(query)

Since we used the grid search to specify two separate cases, we should get two results back:

In [10]:
len(results)

2

We can take a look at the summary for each result

In [11]:
shortest_time_result = results[0]

In [12]:
shortest_time_result["traversal_summary"]

{'distance': 4.23738036063421,
 'distance_unit': 'miles',
 'energy': 0.16431379210115887,
 'energy_unit': 'gallons_gasoline',
 'time': 8.486183959367098,
 'time_unit': 'minutes'}

In [13]:
least_energy_result = results[1]
least_energy_result["traversal_summary"]

{'distance': 4.024996892633556,
 'distance_unit': 'miles',
 'energy': 0.15508906049841378,
 'energy_unit': 'gallons_gasoline',
 'time': 8.536657234467011,
 'time_unit': 'minutes'}

Lastly, let's plot the results to see what each route looks like 

In [14]:
from nrel.routee.compass.plot import plot_route_folium, plot_routes_folium

We can use the `plot_route_folium` function to plot single routes, passing in the `line_kwargs` parameter to customize the folium linestring:

In [15]:
m = plot_route_folium(shortest_time_result, line_kwargs={"color": "blue", "tooltip": "Shortest Time"})
m = plot_route_folium(least_energy_result, line_kwargs={"color": "green", "tooltip": "Least Energy"}, folium_map=m)
m

We can also use the plot_routes_folium function and pass in multiple results. The function will color the routes based on the `value_fn` which takes a single result as an argument. For example, we can tell it to color the routes based on the total energy usage. 

In [16]:
m = plot_routes_folium(results, value_fn=lambda r: r["traversal_summary"]["energy"], color_map="plasma")
m